In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# 1) Charger ton CSV
df = pd.read_csv("BTC_features.csv")

# Convertir open_time en datetime
df["open_time"] = pd.to_datetime(df["open_time"])

# Extraire des features temporelles
df["hour"] = df["open_time"].dt.hour
df["weekday"] = df["open_time"].dt.weekday

# Supprimer colonnes inutiles
drop_cols = ["open_time", "close_time", "ignore"]
df = df.drop(columns=drop_cols, errors="ignore")

# Créer plusieurs horizons
df["future_close_1h"] = df["close"].shift(-1)
df["future_close_3h"] = df["close"].shift(-3)
df["future_close_6h"] = df["close"].shift(-6)

# Créer les cibles binaires
df["target_1h"] = (df["future_close_1h"] > df["close"]).astype(int)
df["target_3h"] = (df["future_close_3h"] > df["close"]).astype(int)
df["target_6h"] = (df["future_close_6h"] > df["close"]).astype(int)

# Supprimer les NaN de fin (car pas de future_close possible)
df = df.dropna().reset_index(drop=True)

# Création des features
feature_cols = [c for c in df.columns if not c.startswith("future_close") and not c.startswith("target")]

Index(['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume',
       'nb_trades', 'taker_buy_base', 'taker_buy_quote', 'sma_7', 'sma_30',
       'sma_50', 'sma_100', 'return', 'volatility_20', 'rsi_14', 'ema_12',
       'ema_26', 'MACD', 'Signal', 'MACD_Hist', 'volume_sma20', 'volume_rel',
       'hour', 'weekday', 'future_close_1h', 'future_close_3h',
       'future_close_6h', 'target_1h', 'target_3h', 'target_6h'],
      dtype='object')

In [20]:
# Normalisation des features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[feature_cols] = scaler.fit_transform(df[feature_cols])
df[feature_cols]

,open,high,low,close,volume,quote_asset_volume,nb_trades,taker_buy_base,taker_buy_quote,sma_7,...,rsi_14,ema_12,ema_26,MACD,Signal,MACD_Hist,volume_sma20,volume_rel,hour,weekday
0,-1.893539,-1.910873,-1.961360,-1.960006,0.425378,0.253619,1.295537,0.153082,0.016896,-1.999246,...,0.253246,-2.010290,-2.058358,0.727614,0.753003,0.067199,-0.074530,0.529623,0.504309,0.487024
1,-1.958215,-1.927586,-1.920676,-1.957143,0.257680,0.106517,0.640438,-0.093245,-0.199598,-1.982622,...,0.284012,-2.001956,-2.050496,0.734950,0.758752,0.073469,-0.069913,0.318711,0.648590,0.487024
2,-1.955354,-1.981576,-1.962838,-1.948809,-0.282683,-0.375047,0.283800,-0.329895,-0.411339,-1.971248,...,0.209513,-1.993620,-2.042598,0.741755,0.764802,0.077112,-0.037421,-0.362836,0.792871,0.487024
3,-1.947025,-1.952928,-1.946996,-1.984639,-0.489922,-0.557231,-0.124850,-0.462546,-0.527410,-1.965898,...,0.145459,-1.992085,-2.037944,0.693716,0.759406,-0.061859,-0.026586,-0.617922,0.937151,0.487024
4,-1.982834,-2.020363,-2.006794,-2.022825,-0.137315,-0.248920,-0.204922,-0.273494,-0.363556,-1.971608,...,0.132370,-1.996669,-2.036468,0.600357,0.735197,-0.290282,0.008102,-0.211713,1.081432,0.487024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2890,-0.093337,-0.120347,-0.089407,-0.087806,-0.363272,-0.367031,-0.447521,-0.288705,-0.292925,-0.169292,...,1.168550,-0.169444,-0.168102,-0.021743,-0.361301,1.026445,-1.069887,0.438842,-1.515619,1.486979
2891,-0.087162,-0.126602,-0.084700,-0.119395,-0.726084,-0.727559,-0.902705,-0.769286,-0.769759,-0.160527,...,0.874315,-0.161225,-0.163900,0.040177,-0.280722,0.982082,-1.086064,-0.437976,-1.371339,1.486979
2892,-0.118732,-0.135966,-0.086547,-0.119554,-0.603844,-0.606263,-0.779926,-0.381186,-0.384939,-0.147798,...,1.044332,-0.154295,-0.160021,0.087213,-0.206237,0.908084,-1.144897,-0.056691,-1.227058,1.486979
2893,-0.118892,-0.158759,-0.162971,-0.182615,-0.437734,-0.443760,-0.655826,-0.574486,-0.578540,-0.142765,...,0.161897,-0.158145,-0.161108,0.044643,-0.155719,0.617067,-1.126448,0.350801,-1.082778,1.486979


In [21]:
def make_sequences_multi(data, seq_len=24, targets=["target_1h","target_3h","target_6h"]):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data.iloc[i:i+seq_len][feature_cols].values)
        y.append(data.iloc[i+seq_len][targets].values)  # 3 targets
    return np.array(X), np.array(y)

SEQ_LEN = 24
X, y = make_sequences_multi(df, seq_len=SEQ_LEN)

print("X shape:", X.shape)  # (n_samples, 24, n_features)
print("y shape:", y.shape)  # (n_samples, 3)

X shape: (2871, 24, 25)
y shape: (2871, 3)


In [22]:
# Split temporel
# Définir les tailles
n = len(X)
train_size = int(n * 0.7)
val_size = int(n * 0.15)

# Découpage temporel
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

print("Train:", X_train.shape, y_train.shape)
print("Val:", X_val.shape, y_val.shape)
print("Test:", X_test.shape, y_test.shape)

Train: (2009, 24, 25) (2009, 3)
Val: (430, 24, 25) (430, 3)
Test: (432, 24, 25) (432, 3)


## Étape 1 — Embedding & Positional Encoding

On commence par créer l'input embedding. L'idée est de faire une couche qui prend les 24h et les 25 features. On doit ajouter la dimension de l'embedding, ici 128, qui transformera mes 25 features en 128 nombres.

`inputs` correspond à un “**placeholder**” qui dit : j’attendrai un batch de shape `(None, 24, 25)`

`x` correspond à un **objet symbolique** (KerasTensor), une sorte de **plan de calcul**.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Création de l'input embedding
inputs = layers.Input(shape=(24, 25))  # 24h, 25 features
x = layers.Dense(128)(inputs)  # projection
x.shape  # (None, 24, 128)

2025-08-31 12:50:31.604271: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1756644636.400133   25878 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9709 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:26:00.0, compute capability: 8.6


(None, 24, 128)

On va coder le positional embedding.

- `max_len=500`: taille maximale de la séquence. Peut aller jusqu'à 500, même si on va utiliser 24 (24h)
- `d_model`: dimmension de l'embedding. Chaque position est représentée par un vecteur de 128 valeurs.
- `pos`: Vecteur de position, allant de 0 à 499 en colonne. Permet d'avoir jusqu'à 500 séquences.
- `i`: index des dimmensions, un vecteur allant de 0 à 127, contenant toutes les dimensions de l'embedding (128).

La formule est la suivante:

$$
PE_{pos, 2i} = \sin{\left( \frac{pos}{10000^{2i/d_{model}}} \right)}, \ PE_{pos, 2i+1} = \cos{\left( \frac{pos}{10000^{2i/d_{model}}} \right)}
$$
Pour les `i` pairs, on utilise la formule de gauche. Pour les `i` impairs, celle de droite. Cette partie est codée à partir de `angle_rates` jusqu'à `pe[:, 1::2]`.

Ensuite, on ajoute la dimension des batchs à `self.pos_encoding`, transformant la matrice en tenseur de dimension 3. La dimension ajouté sers à utiliser des batchs.

Dans le `call`, on prend la taille de la séquence de notre input avec `seq_len`. C'est 24 dans mon cas pour 24h. Enfin, on l'ajoute à notre input pour avoir le résultat.

In [ ]:
import numpy as np

class SinePositionalEncoding(layers.Layer):
    def __init__(self, max_len=500, d_model=128):
        super().__init__()
        pos = np.arange(max_len)[:, None]  # shape (max_len, 1)
        i = np.arange(d_model)[None, :]    # shape (1, d_model)

        angle_rates = 1 / np.power(10000, (2*(i//2)) / np.float32(d_model))
        angles = pos * angle_rates

        # tableau (max_len, d_model)
        pe = np.zeros((max_len, d_model))
        pe[:, 0::2] = np.sin(angles[:, 0::2])  # pairs
        pe[:, 1::2] = np.cos(angles[:, 1::2])  # impairs

        # on stocke comme constante TensorFlow
        self.pos_encoding = tf.constant(pe[None, ...], dtype=tf.float32)  # (1, max_len, d_model)

    def call(self, x):
        seq_len = tf.shape(x)[1]
        return x + self.pos_encoding[:, :seq_len, :]